In [1]:
!pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.2 MB/s eta 0:00:00


QC_SUBSET_CONF1(NO TUNING)

In [2]:
pip install medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=009b33b0936479312e4b7e0e85c050e9e0d8e3b02a2edcc5e0d2c8fea13a885f
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire


In [3]:
from torchvision import transforms

QC AVG ACC PREC F1 COSINE

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import CLIPProcessor, CLIPModel
import pennylane as qml
import numpy as np
from medmnist import INFO
import medmnist
import copy
from sklearn.metrics import precision_score, f1_score

torch.manual_seed(42)
np.random.seed(42)

n_qubits = 8
q_depth = 4
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

weight_shapes = {"weights": (q_depth, n_qubits)}
qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)

class QuantumCLIP(nn.Module):
    def __init__(self, latent_dim=128, n_qubits=8):
        super(QuantumCLIP, self).__init__()

        self.text_encoder = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.text_projection = nn.Linear(self.text_encoder.config.text_config.hidden_size, latent_dim, bias=True)

        # Downsampling for images
        self.downsample = nn.Linear(28 * 28, n_qubits, bias=True)


        self.quantum_layer = qlayer


        self.post_net = nn.Linear(n_qubits, latent_dim, bias=True)

    def forward(self, text_inputs, images):
        text_embeddings = self.text_encoder.get_text_features(**text_inputs)
        text_latent = self.text_projection(text_embeddings)

        images = images.view(images.size(0), -1)
        images = self.downsample(images)
        q_out = self.quantum_layer(images)
        image_latent = self.post_net(q_out)

        return text_latent, image_latent


class MedMNISTDataset(Dataset):
    def __init__(self, split, transform=None):
        info = INFO["pathmnist"]
        self.data_flag = "pathmnist"
        self.task = info["task"]
        self.n_channels = info["n_channels"]
        self.n_classes = len(info["label"])
        self.class_names = {int(k): v for k, v in info["label"].items()}

        DataClass = getattr(medmnist, info["python_class"])
        self.data = DataClass(split=split, download=True)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img, label = self.data[idx]
        if isinstance(label, np.ndarray):
            label = label.item()
        if self.transform:
            img = self.transform(img)
        text = f"This is a {self.class_names[label]}."
        return img, text, label


def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, num_epochs):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

       validation_metrics = {
        "loss": [],
        "accuracy": [],
        "precision": [],
        "f1": [],
        "cosine_similarity": []
    }

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print("-" * 10)

        for phase in ["train", "validation"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            total_loss = 0.0
            total_corrects = 0
            total_samples = 0
            all_preds = []
            all_labels = []
            all_cosine_sims = []

            for images, texts, labels in dataloaders[phase]:
                images, labels = images.to(device), labels.to(device)
                text_inputs = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")(
                    text=texts, return_tensors="pt", padding=True, truncation=True
                )
                text_inputs = {k: v.to(device) for k, v in text_inputs.items()}

                optimizer.zero_grad()


                with torch.set_grad_enabled(phase == "train"):
                    text_latent, image_latent = model(text_inputs, images)
                    logits = torch.matmul(F.normalize(text_latent), F.normalize(image_latent).T)
                    targets = torch.arange(len(logits)).to(device)
                    loss = criterion(logits, targets)

                    _, preds = torch.max(logits, 1)
                    cosine_similarity = F.cosine_similarity(text_latent, image_latent, dim=1)

                    if phase == "train":
                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                        optimizer.step()


                total_loss += loss.item() * images.size(0)  # Scale loss by batch size
                total_corrects += torch.sum(preds == labels.data).item()
                total_samples += labels.size(0)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_cosine_sims.extend(cosine_similarity.detach().cpu().numpy())


            epoch_loss = total_loss / total_samples
            epoch_acc = total_corrects / total_samples
            epoch_f1 = f1_score(all_labels, all_preds, average="weighted")
            epoch_precision = precision_score(all_labels, all_preds, average="weighted")

            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}, "
                  f"F1-Score: {epoch_f1:.4f}, Precision: {epoch_precision:.4f}")


            if phase == "validation":
                validation_metrics["loss"].append(epoch_loss)
                validation_metrics["accuracy"].append(epoch_acc)
                validation_metrics["precision"].append(epoch_precision)
                validation_metrics["f1"].append(epoch_f1)
                validation_metrics["cosine_similarity"].extend(all_cosine_sims)


                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

            if phase == "train":
                scheduler.step()


    avg_loss = np.mean(validation_metrics["loss"])
    avg_accuracy = np.mean(validation_metrics["accuracy"])
    avg_precision = np.mean(validation_metrics["precision"])
    avg_f1 = np.mean(validation_metrics["f1"])
    avg_cosine_similarity = np.mean(validation_metrics["cosine_similarity"])

    print("\nTraining Complete!")
    print(f"Validation Average Loss: {avg_loss:.4f}")
    print(f"Validation Average Accuracy: {avg_accuracy:.4f}")
    print(f"Validation Average F1-Score: {avg_f1:.4f}")
    print(f"Validation Average Precision: {avg_precision:.4f}")
    print(f"Validation Average Cosine Similarity: {avg_cosine_similarity:.4f}")

    model.load_state_dict(best_model_wts)
    return model




# Main Script
if __name__ == "__main__":
    transform = transforms.Compose([
        transforms.Resize((28, 28)),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])

    train_dataset = MedMNISTDataset(split="train", transform=transform)
    test_dataset = MedMNISTDataset(split="test", transform=transform)

    train_subset, _ = random_split(train_dataset, [500, len(train_dataset) - 500])
    test_subset, _ = random_split(test_dataset, [100, len(test_dataset) - 100])

    dataset_sizes = {"train": len(train_subset), "validation": len(test_subset)}

    dataloaders = {
        "train": DataLoader(train_subset, batch_size=16, shuffle=True),
        "validation": DataLoader(test_subset, batch_size=16, shuffle=False)
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = QuantumCLIP().to(device)
    for param in model.text_encoder.parameters():
        param.requires_grad = True

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    model = train_model(model, dataloaders, dataset_sizes, criterion, optimizer, exp_lr_scheduler, num_epochs=3)



100%|██████████| 206M/206M [00:16<00:00, 12.6MB/s]


Using downloaded and verified file: /root/.medmnist/pathmnist.npz


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

Validation Loss: 2.7246, Accuracy: 0.1100, F1-Score: 0.0962, Precision: 0.0897
Epoch 4/5
----------
Train Loss: 2.7598, Accuracy: 0.0560, F1-Score: 0.0583, Precision: 0.0718


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validation Loss: 2.7020, Accuracy: 0.1100, F1-Score: 0.1003, Precision: 0.0929
Epoch 5/5
----------
Train Loss: 2.6977, Accuracy: 0.0700, F1-Score: 0.0924, Precision: 0.1870


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validation Loss: 2.7484, Accuracy: 0.0700, F1-Score: 0.0745, Precision: 0.0944

Training Complete!
Validation Average Loss: 2.7023
Validation Average Accuracy: 0.0800
Validation Average F1-Score: 0.0792
Validation Average Precision: 0.0899
Validation Average Cosine Similarity: 0.0786


TRAIN_TEST_VAL


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import CLIPProcessor, CLIPModel
import pennylane as qml
import numpy as np
from medmnist import INFO
import medmnist
import copy
from sklearn.metrics import precision_score, f1_score


torch.manual_seed(42)
np.random.seed(42)


n_qubits = 8
q_depth = 4
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]


weight_shapes = {"weights": (q_depth, n_qubits)}
qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)


class QuantumCLIP(nn.Module):
    def __init__(self, latent_dim=128, n_qubits=8):
        super(QuantumCLIP, self).__init__()

        self.text_encoder = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.text_projection = nn.Linear(self.text_encoder.config.text_config.hidden_size, latent_dim, bias=True)

        # Downsampling for images
        self.downsample = nn.Linear(28 * 28, n_qubits, bias=True)


        self.quantum_layer = qml.qnn.TorchLayer(qnode, weight_shapes)


        self.post_net = nn.Linear(n_qubits, latent_dim, bias=True)

    def forward(self, text_inputs, images):
        text_embeddings = self.text_encoder.get_text_features(**text_inputs)
        text_latent = self.text_projection(text_embeddings)

        images = images.view(images.size(0), -1)
        images = self.downsample(images)
        q_out = self.quantum_layer(images)
        image_latent = self.post_net(q_out)

        return text_latent, image_latent





class MedMNISTDataset(Dataset):
    def __init__(self, split, transform=None):
        info = INFO["pathmnist"]
        self.data_flag = "pathmnist"
        self.task = info["task"]
        self.n_channels = info["n_channels"]
        self.n_classes = len(info["label"])
        self.class_names = {int(k): v for k, v in info["label"].items()}

        DataClass = getattr(medmnist, info["python_class"])
        self.data = DataClass(split=split, download=True)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img, label = self.data[idx]
        if isinstance(label, np.ndarray):
            label = label.item()
        if self.transform:
            img = self.transform(img)
        text = f"This is a {self.class_names[label]}."
        return img, text, label


from sklearn.metrics import f1_score, precision_score

def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, num_epochs):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0


    validation_metrics = {
        "loss": [],
        "accuracy": [],
        "precision": [],
        "f1": [],
        "cosine_similarity": []
    }

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print("-" * 10)

        for phase in ["train", "validation"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            total_loss = 0.0
            total_corrects = 0
            total_samples = 0
            all_preds = []
            all_labels = []
            all_cosine_sims = []

            for images, texts, labels in dataloaders[phase]:
                images, labels = images.to(device), labels.to(device)
                text_inputs = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")(
                    text=texts, return_tensors="pt", padding=True, truncation=True
                )
                text_inputs = {k: v.to(device) for k, v in text_inputs.items()}

                optimizer.zero_grad()


                with torch.set_grad_enabled(phase == "train"):
                    text_latent, image_latent = model(text_inputs, images)
                    logits = torch.matmul(F.normalize(text_latent), F.normalize(image_latent).T)
                    targets = torch.arange(len(logits)).to(device)
                    loss = criterion(logits, targets)

                    if phase == "train":
                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                        optimizer.step()

                    _, preds = torch.max(logits, 1)
                    cosine_similarity = F.cosine_similarity(text_latent, image_latent, dim=1)


                    total_loss += loss.item() * images.size(0)  # Scale loss by batch size
                    total_corrects += torch.sum(preds == targets.data).item()
                    total_samples += labels.size(0)
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(labels.cpu().numpy())
                    all_cosine_sims.extend(cosine_similarity.detach().cpu().numpy())


            epoch_loss = total_loss / total_samples
            epoch_acc = total_corrects / total_samples
            epoch_f1 = f1_score(all_labels, all_preds, average="weighted")
            epoch_precision = precision_score(all_labels, all_preds, average="weighted")

            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}, "
                  f"F1-Score: {epoch_f1:.4f}, Precision: {epoch_precision:.4f}")


            if phase == "validation":
                validation_metrics["loss"].append(epoch_loss)
                validation_metrics["accuracy"].append(epoch_acc)
                validation_metrics["precision"].append(epoch_precision)
                validation_metrics["f1"].append(epoch_f1)
                validation_metrics["cosine_similarity"].extend(all_cosine_sims)


                if epoch_acc > best_val_acc:
                    best_val_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

            if phase == "train":
                scheduler.step()


    avg_val_loss = np.mean(validation_metrics["loss"])
    avg_val_accuracy = np.mean(validation_metrics["accuracy"])
    avg_val_precision = np.mean(validation_metrics["precision"])
    avg_val_f1 = np.mean(validation_metrics["f1"])
    avg_val_cosine_similarity = np.mean(validation_metrics["cosine_similarity"])

    print("\nTraining Complete!")
    print(f"Validation Average Loss: {avg_val_loss:.4f}")
    print(f"Validation Average Accuracy: {avg_val_accuracy:.4f}")
    print(f"Validation Average F1-Score: {avg_val_f1:.4f}")
    print(f"Validation Average Precision: {avg_val_precision:.4f}")
    print(f"Validation Average Cosine Similarity: {avg_val_cosine_similarity:.4f}")

    model.load_state_dict(best_model_wts)
    return model


def evaluate_model(model, dataloader, device, criterion):
    model.eval()
    total_loss = 0.0
    total_corrects = 0
    total_samples = 0
    all_preds = []
    all_labels = []
    all_cosine_sims = []

    with torch.no_grad():
        for images, texts, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            text_inputs = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")(
                text=texts, return_tensors="pt", padding=True, truncation=True
            )
            text_inputs = {k: v.to(device) for k, v in text_inputs.items()}

            text_latent, image_latent = model(text_inputs, images)
            logits = torch.matmul(F.normalize(text_latent), F.normalize(image_latent).T)
            targets = torch.arange(len(logits)).to(device)
            loss = criterion(logits, targets)

            _, preds = torch.max(logits, 1)
            cosine_similarity = F.cosine_similarity(text_latent, image_latent, dim=1)


            total_loss += loss.item() * images.size(0)
            total_corrects += torch.sum(preds == targets.data).item()
            total_samples += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_cosine_sims.extend(cosine_similarity.detach().cpu().numpy())


    avg_loss = total_loss / total_samples
    avg_accuracy = total_corrects / total_samples
    avg_f1 = f1_score(all_labels, all_preds, average="weighted")
    avg_precision = precision_score(all_labels, all_preds, average="weighted")
    avg_cosine_similarity = np.mean(all_cosine_sims)

    print("\nTest Set Metrics:")
    print(f"Test Loss: {avg_loss:.4f}")
    print(f"Test Accuracy: {avg_accuracy:.4f}")
    print(f"Test F1-Score: {avg_f1:.4f}")
    print(f"Test Precision: {avg_precision:.4f}")
    print(f"Test Cosine Similarity: {avg_cosine_similarity:.4f}")

    return avg_loss, avg_accuracy, avg_f1, avg_precision, avg_cosine_similarity


if __name__ == "__main__":
    transform = transforms.Compose([
        transforms.Resize((28, 28)),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])


    train_dataset = MedMNISTDataset(split="train", transform=transform)
    test_dataset = MedMNISTDataset(split="test", transform=transform)


    train_size = 1000
    val_size = 100
    test_size = 200


    total_train_size = train_size + val_size
    train_val_subset, _ = random_split(train_dataset, [total_train_size, len(train_dataset) - total_train_size])
    train_subset, val_subset = random_split(train_val_subset, [train_size, val_size])


    test_subset, _ = random_split(test_dataset, [test_size, len(test_dataset) - test_size])


    train_loader = DataLoader(train_subset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=16, shuffle=False)
    test_loader = DataLoader(test_subset, batch_size=16, shuffle=False)


    dataloaders = {
        "train": train_loader,
        "validation": val_loader
    }
    dataset_sizes = {
        "train": len(train_subset),
        "validation": len(val_subset)
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



    model = QuantumCLIP(latent_dim=128, n_qubits=8).to(device)
    for param in model.text_encoder.parameters():
        param.requires_grad = True

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)


    model = train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, num_epochs=5)


    evaluate_model(model, test_loader, device, criterion)


100%|██████████| 206M/206M [00:12<00:00, 16.7MB/s]


Using downloaded and verified file: /root/.medmnist/pathmnist.npz


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

Validation Loss: 2.5955, Accuracy: 0.1200, F1-Score: 0.0858, Precision: 0.1026
Epoch 3/5
----------
Train Loss: 2.6298, Accuracy: 0.1200, F1-Score: 0.0741, Precision: 0.1146
Validation Loss: 2.6656, Accuracy: 0.1400, F1-Score: 0.1404, Precision: 0.2228
Epoch 4/5
----------
Train Loss: 2.5811, Accuracy: 0.1540, F1-Score: 0.0790, Precision: 0.1191


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validation Loss: 2.4893, Accuracy: 0.1900, F1-Score: 0.1121, Precision: 0.1726
Epoch 5/5
----------
Train Loss: 2.5054, Accuracy: 0.1730, F1-Score: 0.0806, Precision: 0.1077


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validation Loss: 2.5284, Accuracy: 0.1700, F1-Score: 0.0795, Precision: 0.1368

Training Complete!
Validation Average Loss: 2.5933
Validation Average Accuracy: 0.1500
Validation Average F1-Score: 0.0912
Validation Average Precision: 0.1394
Validation Average Cosine Similarity: 0.3457

Test Set Metrics:
Test Loss: 2.4594
Test Accuracy: 0.1650
Test F1-Score: 0.0518
Test Precision: 0.0791
Test Cosine Similarity: 0.4734
